In [1]:
import random
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from dataclasses import dataclass

In [2]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed);

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device {device}')

Using device cuda


### Prepare the data

In [4]:
class ImpurityDataset(Dataset):
    
    def __init__(self, dataframe, fixed_features, labels, feature_scaler=None, label_r_scaler=None, label_i_scaler=None, device=None):
        assert len(labels) % 2 == 0
        
        self.fixed_features = fixed_features
        self.labels = labels
        self.n_samples = len(dataframe)
        
        self.output_length = 2
        self.input_length = len(fixed_features) + len(labels) - self.output_length
        self.sequence_length = len(labels) // self.output_length

        df_features = dataframe[fixed_features]
        df_labels = dataframe[labels]
        
        if feature_scaler is not None and label_r_scaler is not None and label_i_scaler is not None:
            xs = feature_scaler.transform(df_features)
            #ys = df_labels.values
            
            ys_r = label_r_scaler.transform(df_labels[df_labels.columns[::2]])
            ys_i = label_i_scaler.transform(df_labels[df_labels.columns[1::2]])
            
            ys = np.empty((ys_r.shape[0], ys_r.shape[1] * 2), dtype=ys_r.dtype)
            ys[:, ::2] = ys_r  # fill even indices with real parts
            ys[:, 1::2] = ys_i  # fill odd indices with imaginary parts
        else:
            xs = df_features.values
            ys = df_labels.values

        feature_data = np.zeros((self.n_samples, self.sequence_length, self.input_length))
        label_data = np.zeros((self.n_samples, self.sequence_length, self.output_length))
        
        for i in range(self.n_samples):
            for j in range(self.sequence_length):
                xi = xs[i]
                xj = ys[i][0:j*2]
                y = ys[i][j*2:j*2+2]
        
                features = np.concatenate([xi, xj], axis=0)
                pad_width = self.input_length - len(features)
                feature_data[i, j, :] = np.pad(features, (0, pad_width))
                label_data[i, j, :] = y

        self.feature_data = torch.tensor(feature_data, dtype=torch.float).to(device)
        self.label_data = torch.tensor(label_data, dtype=torch.float).to(device)

    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        return self.feature_data[idx], self.label_data[idx]

In [5]:
def compute_scalers(dataframe, fixed_features, labels, test_size=0.1, random_state=None):
    train_df, _ = train_test_split(dataframe, test_size=test_size, random_state=random_state)
    df_features = train_df[fixed_features]
    df_labels = train_df[labels]
    
    feature_scaler = StandardScaler()
    label_r_scaler = StandardScaler()
    label_i_scaler = StandardScaler()

    feature_scaler.fit(df_features)
    label_r_scaler.fit(df_labels[df_labels.columns[::2]]) # get only real columns
    label_i_scaler.fit(df_labels[df_labels.columns[1::2]]) # get only imaginary columns

    return feature_scaler, label_r_scaler, label_i_scaler

In [6]:
file_path = 'data/20230825_144318_10k_EVDoubExp-TExp-wmax5-sparse-hyb_with_perturbation.csv'

fixed_features = ['beta', 'E1', 'E2', 'E3', 'V1', 'V2', 'V3', 'ReFso1', 'ImFso1', 'ReFso3', 'ImFso3', 'ReFso5', 'ImFso5', 'ReFso7', 'ImFso7', 'ReFso9', 'ImFso9', 'ReFso11', 'ImFso11', 'ReFso13', 'ImFso13', 'ReFso15', 'ImFso15', 'ReFso17', 'ImFso17', 'ReFso19', 'ImFso19', 'ReFso21', 'ImFso21', 'ReFso23', 'ImFso23', 'ReFso25', 'ImFso25', 'ReFso29', 'ImFso29', 'ReFso33', 'ImFso33', 'ReFso37', 'ImFso37', 'ReFso43', 'ImFso43', 'ReFso49', 'ImFso49', 'ReFso57', 'ImFso57', 'ReFso69', 'ImFso69', 'ReFso83', 'ImFso83', 'ReFso101', 'ImFso101', 'ReFso127', 'ImFso127', 'ReFso165', 'ImFso165', 'ReFso237', 'ImFso237', 'ReFso399', 'ImFso399', 'ReFso1207', 'ImFso1207']
labels = ['ReSf1', 'ImSf1', 'ReSf3', 'ImSf3', 'ReSf5', 'ImSf5', 'ReSf7', 'ImSf7', 'ReSf9', 'ImSf9', 'ReSf11', 'ImSf11', 'ReSf13', 'ImSf13', 'ReSf15', 'ImSf15', 'ReSf17', 'ImSf17', 'ReSf19', 'ImSf19', 'ReSf21', 'ImSf21', 'ReSf23', 'ImSf23', 'ReSf25', 'ImSf25', 'ReSf29', 'ImSf29', 'ReSf33', 'ImSf33', 'ReSf37', 'ImSf37', 'ReSf43', 'ImSf43', 'ReSf49', 'ImSf49', 'ReSf57', 'ImSf57', 'ReSf69', 'ImSf69', 'ReSf83', 'ImSf83', 'ReSf101', 'ImSf101', 'ReSf127', 'ImSf127', 'ReSf165', 'ImSf165', 'ReSf237', 'ImSf237', 'ReSf399', 'ImSf399', 'ReSf1207', 'ImSf1207']

df = pd.read_csv(file_path, skiprows=4) # we skip the first four lines, because they are just metadata
df = df[fixed_features + labels]

validation_size = 0.1 # 90% training, 10% for validation

feature_scaler, label_r_scaler, label_i_scaler = compute_scalers(df, fixed_features, labels, validation_size, seed) # make sure we use the same seed, otherwise the two splits differ!
#dataset = ImpurityDataset(df, fixed_features, labels, device=device)
dataset = ImpurityDataset(df, fixed_features, labels, feature_scaler, label_r_scaler, label_i_scaler, device)  

indices = list(range(len(dataset)))
train_indices, val_indices = train_test_split(indices, test_size=validation_size, random_state=seed)  # make sure we use the same seed, otherwise the two splits differ!

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)

In [7]:
dataset.__getitem__(0)[0].shape

torch.Size([27, 113])

### Define the model

In [8]:
class PositionalEncodingL(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=27):
        super(PositionalEncodingL, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.positional_embedding = nn.Parameter(torch.zeros(max_len, d_model))
    
    def forward(self, x):
        # x: [batch_size, seq_len, d_model]
        batch_size, seq_len, d_model = x.shape
        
        # Use the first 'seq_len' positions from the positional_embedding
        # Expand the positional embeddings to match the batch size and add them to the input
        position_encoded = self.positional_embedding[:, :].unsqueeze(0).expand(batch_size, -1, -1)
        x = x + position_encoded
        return self.dropout(x)

In [9]:
class PositionalEncodingF(torch.nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=27):
        super(PositionalEncodingF, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [10]:
@dataclass
class ModelConfig:
    input_dim: int
    output_dim: int
    sequence_length: int
    
    d_model: int
    nhead: int
    num_layers: int
    dim_feedforward: int
    
    dropout: float
    activation: str
    bias: bool

class AutoregressiveTransformer(nn.Module):
    
    def __init__(self, config, device):
        super(AutoregressiveTransformer, self).__init__()

        self.config = config

        #self.input_projection = nn.Linear(config.input_dim, config.d_model)
        self.input_projection = nn.Linear(config.input_dim, config.d_model)
        
        self.positional_encoding = PositionalEncodingL(config.d_model, dropout=config.dropout, max_len=config.sequence_length)
        
        decoder_layer = nn.TransformerDecoderLayer(
            d_model=config.d_model, 
            nhead=config.nhead, 
            dim_feedforward=config.dim_feedforward, 
            dropout=config.dropout,
            activation=config.activation, 
            batch_first=True, 
            norm_first=True, 
            bias=config.bias
        )
        
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=config.num_layers)
        
        self.output_layer = nn.Linear(config.d_model, config.output_dim)
        
        self.att_mask = self.generate_mask(config.sequence_length, device)
        
    def forward(self, x):        
        x = self.input_projection(x)        
        x = self.positional_encoding(x)
        
        output = self.transformer_decoder(x, x, tgt_mask=self.att_mask)
        output = self.output_layer(output)
        
        return output

    def generate_mask(self, sequence_length, device):
        mask = torch.zeros((sequence_length, sequence_length), device=device)
        mask = torch.triu(mask.fill_(float('-inf')), diagonal=1)
            
        return mask

### Initialize the model

In [11]:
config = ModelConfig(
    input_dim = len(fixed_features) + len(labels) - 2,
    output_dim = 2,
    sequence_length = 27,
    
    d_model = 256,
    nhead = 4,
    num_layers = 2,
    dim_feedforward = 256 * 4,
    
    dropout = 0.1,
    activation = 'gelu',
    bias = True
)

model = AutoregressiveTransformer(config, device).to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [12]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    
    for inputs, targets in train_loader:
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    return total_loss / len(train_loader)

In [13]:
def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        
        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
    
    return total_loss / len(val_loader)

### Train the model

In [14]:
num_epochs = 200

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss = validate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

C:\Users\Patri\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 1, Train Loss: 0.120294, Val Loss: 0.031402
Epoch 2, Train Loss: 0.037691, Val Loss: 0.017858
Epoch 3, Train Loss: 0.026273, Val Loss: 0.018781
Epoch 4, Train Loss: 0.019862, Val Loss: 0.008970
Epoch 5, Train Loss: 0.015695, Val Loss: 0.006983
Epoch 6, Train Loss: 0.014220, Val Loss: 0.009028
Epoch 7, Train Loss: 0.010983, Val Loss: 0.004903
Epoch 8, Train Loss: 0.010199, Val Loss: 0.005700
Epoch 9, Train Loss: 0.009283, Val Loss: 0.006580
Epoch 10, Train Loss: 0.008007, Val Loss: 0.003460
Epoch 11, Train Loss: 0.007607, Val Loss: 0.004140
Epoch 12, Train Loss: 0.006444, Val Loss: 0.004266
Epoch 13, Train Loss: 0.006263, Val Loss: 0.004156
Epoch 14, Train Loss: 0.006214, Val Loss: 0.003053
Epoch 15, Train Loss: 0.005468, Val Loss: 0.003391
Epoch 16, Train Loss: 0.005732, Val Loss: 0.002381
Epoch 17, Train Loss: 0.005035, Val Loss: 0.002134
Epoch 18, Train Loss: 0.004894, Val Loss: 0.002321
Epoch 19, Train Loss: 0.004959, Val Loss: 0.002775
Epoch 20, Train Loss: 0.004279, Val Loss

### Manual model validation

In [15]:
def validate_mape(model, val_loader, scaler_r, scaler_i, device, epsilon=1e-8):
    model.eval()
    total_loss = 0.0
    
    with torch.no_grad():
        
        for inputs, targets in val_loader:
            
            outputs = model(inputs)
            
            outputs = torch.tensor(np.array([reverse_tranform_output(o, scaler_r, scaler_i) for o in outputs]))
            targets = torch.tensor(np.array([reverse_tranform_output(t, scaler_r, scaler_i) for t in targets]))

            ape = torch.abs((targets - outputs) / (targets + epsilon))
            mape = torch.mean(ape) * 100
                    
            total_loss += mape.item()
    
    return total_loss / len(val_loader)

In [16]:
def reverse_tranform_output(data, scaler_r, scaler_i):
    data = data.cpu().numpy()

    r = scaler_r.inverse_transform(data[:, ::2].reshape(1, -1)).reshape(-1, 1)
    i = scaler_i.inverse_transform(data[:, 1::2].reshape(1, -1)).reshape(-1, 1)
            
    return np.concatenate((r, i), axis=1)

In [17]:
validate_mape(model, val_loader, label_r_scaler, label_i_scaler, device)

2.5195905305445194

### Sample from the model

In [18]:
def sample_from_model(model, input, sequence_length, fixed_feature_len, fixed_labels_len, max_features, device):
    model.eval()

    fixed_features = input[:fixed_feature_len]
    fixed_labels = input[fixed_feature_len:fixed_labels_len*2 + fixed_feature_len]
    
    assert len(fixed_labels) % 2 == 0
    provided_sequences = len(fixed_labels) // 2
    
    initial_input = torch.zeros(1, sequence_length, max_features)
    initial_input[0, 0, :fixed_feature_len] = fixed_features

    outputs = torch.zeros(sequence_length, 2, device = device)
    
    for i in range(1, provided_sequences + 1):
        labels = fixed_labels[(i-1)*2:2*i]
        initial_input[0, i, :2*i + fixed_feature_len] = initial_input[0, i-1, :2*i + fixed_feature_len]
        initial_input[0, i, 2*(i-1) + fixed_feature_len:2*i + fixed_feature_len] = labels
        outputs[i-1, :] = labels

    current_input = initial_input.to(device)

    
    with torch.no_grad():
    
        for i in range(fixed_labels_len + 1, sequence_length + 1):

            output = model(current_input)
            predictions = output[0, -1, -2:]

            outputs[i-1, :] = predictions
            
            if i == sequence_length:
                break
            
            position_to_insert = i * 2 + fixed_feature_len
            current_input[0, i, :position_to_insert-2] = current_input[0, i-1, :position_to_insert-2]
            current_input[0, i, position_to_insert-2:position_to_insert] = predictions

    return outputs

In [19]:
def convert(data):
    data = data.cpu().numpy()
            
    r = label_r_scaler.inverse_transform(data[::2].reshape(1, -1)).reshape(-1, 1)
    i = label_i_scaler.inverse_transform(data[1::2].reshape(1, -1)).reshape(-1, 1)
            
    return np.concatenate((r, i), axis=1)

In [20]:
def calculate_mape(true_values, predictions, epsilon=1e-8):
    true_values = np.array(true_values)
    predictions = np.array(predictions)
    
    mape = np.mean(np.abs((true_values - predictions) / (true_values + epsilon))) * 100
    return mape

In [21]:
n_samples = 100 #train_loader.dataset.dataset.n_samples 
sequence_length = 27
fixed_labels_len = 0
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

mapes = []

for idx in range(n_samples):
    input = val_loader.dataset[idx][0][fixed_labels_len]
    output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
    
    target = convert(val_loader.dataset[idx][1].reshape(-1))
    output = convert(output.reshape(-1))
    
    #print(output)
    #print(target)
    
    mapes.append(calculate_mape(target, output))

    if (idx+1) % 100 == 0:
        print(idx+1)

np.mean(mapes)

100


55.94038159400225

In [22]:
n_samples = 1000
sequence_length = 27
fixed_features_len = len(fixed_features)
max_features = fixed_features_len + len(labels) - 2

total_mapes = []

for j in range(sequence_length):
    fixed_labels_len = j
    
    mapes = []
    
    for idx in range(n_samples):
        input = val_loader.dataset[idx][0][fixed_labels_len]
        output = sample_from_model(model, input, sequence_length, fixed_features_len, fixed_labels_len, max_features, device)
        
        target = convert(val_loader.dataset[idx][1].reshape(-1))
        output = convert(output.reshape(-1))
        
        mapes.append(calculate_mape(target, output))
    
        if (idx+1) % 1000 == 0:
            print(idx+1)

    total_mapes.append(np.mean(mapes))

print(total_mapes)

1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
1000
[71.54995989352464, 47.707403323426846, 40.427570118010046, 35.960545524954796, 32.53722260370851, 30.02346686720848, 27.83183875977993, 25.83194867428392, 24.11481358949095, 22.403499990142883, 20.769613812863827, 19.317979664355516, 17.878177537117153, 16.493516808375716, 15.059988837130367, 13.68213702137582, 12.377278786059469, 11.072180250007658, 9.789127199398354, 8.512608890258708, 7.24103077116888, 5.968304853467271, 4.722670222213492, 3.502867814945057, 2.3187418103741946, 1.1715450496645645, 0.06796834189776746]
